# Projeto Grupo BT-G3


## **Integrantes do grupo**
- Daniel Barzilai
- Larissa Carvalho
- Maria Luisa Maia
- Pedro Rezende
- Rafael Moritz
- Vitor Oliveira

<center><img src="https://www.inteli.edu.br/wp-content/uploads/2021/08/20172028/marca_1-2.png" width="50%" height="50%"/></center>

<h1 align='center'><b>IA para Marketing: Monitoramento de campanhas utilizando processamento de linguagem natural (PLN)<b></h1>

<center><img src="https://upload.wikimedia.org/wikipedia/commons/c/c2/Btg-logo-blue.svg" width="50%" height="50%"/></center>

<h2 align='center'>O Banco BTG Pactual enfrenta um desafio na área de Marketing em entender as necessidades e demandas dos clientes de maneira fácil e rápida nas redes sociais. A solução proposta para esse problema foi o desenvolvimento de uma Inteligência Artificial utilizando processamento de linguagem natural (PLN), capaz de monitorar as campanhas de marketing, voltadas para o Instagram. O objetivo principal dessa solução é rastrear os dados em tempo real, analisar e interpretar as mensagens e comentários enviados pelos clientes na rede social, a fim de identificar as necessidades e demandas de forma precisa e eficiente.</h2>

---

# Sobre os dados

Esse projeto está utilizando dados coletados e tratados pela equipe de Automation do BTG Pactual, o qual disponibilizou o dataset. Com base nas informações dispostas nesse dataset, realizaremos insights a cerca dos comentários feitos nos posts do Instagram do próprio banco. Vale lembrar que os dados estão anonimizados e resguardados para manter a privacidade e ética com os usuários e com o banco.

# 1. Instalação / Setup

Para o início do projeto, fizemos o desenvolvimento no Google Colab, por isso temos uma célula de conexão com o Google Drive, para poder acessar os dados. Caso seja rodado no Jupyter Notebook, precisará do dataset baixado.

In [27]:
#Conectar com o Google Drive

from google.colab import drive
drive.mount('/content/drive')

#Conectando o ambiente ao Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Aqui nós fazemos as importações para tratamento dos dados, pré-processamento dos dados e modelamento do Bag of Words. 

## pips

In [28]:
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
!pip install wordcloud

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Imports

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re

from sklearn.feature_extraction.text import CountVectorizer
import ast
from keras.preprocessing.text import Tokenizer
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential 
from keras.layers import Dense, Embedding, GlobalMaxPooling1D, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from keras.utils import pad_sequences

import nltk
import spacy
import gensim
import pickle
from scipy.spatial.distance import cosine
from gensim.models import KeyedVectors

O desenvolvimento da rede neural seguiu o tutorial abaixo:
https://medium.com/@jvsavietto6/classificando-texto-com-redes-neurais-artificiais-150ef448b13d

# 2. Entendimento e Tratamento dos Dados

Rodando o dataset, para analisar seu conteúdo:

In [34]:
df = pd.read_csv('/content/drive/MyDrive/Módulo 6/Sprint 3 - Projeto/nova_base_tratada (1)')
df

,Unnamed: 0,autor,sentimento,texto_tratado
0,0,winthegame_of,1,"['alvarez', 'marsal', 'estar', 'conosco', 'spo..."
1,1,marta_bego,1,"['btgpactual', 'with', 'makerepost', 'entender..."
2,2,lmviapiana,2,"['minuto', 'touro', 'ouro']"
3,3,vanilson_dos,1,['sim']
4,4,ricktolledo,2,"['querer', 'saber', 'banking', 'próprio', 'adm..."
...,...,...,...,...
9202,9472,perspectiveinvestimentos,2,"['excelente', 'explicação']"
9203,9473,eduardocolares,2,"['atendar', 'telefone', 'amor', 'deus']"
9204,9474,danielucm,2,"['saber', 'qual', 'grande', 'fiis', 'mercado',..."
9205,9475,amgcapitalinvest,1,"['erro', 'financeiro', 'eliminar', 'antes', 'a..."


In [35]:
df.columns

Index(['Unnamed: 0', 'autor', 'sentimento', 'texto_tratado'], dtype='object')

In [36]:
df = df.drop(['Unnamed: 0'], axis=1)
df

,autor,sentimento,texto_tratado
0,winthegame_of,1,"['alvarez', 'marsal', 'estar', 'conosco', 'spo..."
1,marta_bego,1,"['btgpactual', 'with', 'makerepost', 'entender..."
2,lmviapiana,2,"['minuto', 'touro', 'ouro']"
3,vanilson_dos,1,['sim']
4,ricktolledo,2,"['querer', 'saber', 'banking', 'próprio', 'adm..."
...,...,...,...
9202,perspectiveinvestimentos,2,"['excelente', 'explicação']"
9203,eduardocolares,2,"['atendar', 'telefone', 'amor', 'deus']"
9204,danielucm,2,"['saber', 'qual', 'grande', 'fiis', 'mercado',..."
9205,amgcapitalinvest,1,"['erro', 'financeiro', 'eliminar', 'antes', 'a..."


In [37]:
df['texto_tratado']

0       ['alvarez', 'marsal', 'estar', 'conosco', 'spo...
1       ['btgpactual', 'with', 'makerepost', 'entender...
2                             ['minuto', 'touro', 'ouro']
3                                                 ['sim']
4       ['querer', 'saber', 'banking', 'próprio', 'adm...
                              ...                        
9202                          ['excelente', 'explicação']
9203              ['atendar', 'telefone', 'amor', 'deus']
9204    ['saber', 'qual', 'grande', 'fiis', 'mercado',...
9205    ['erro', 'financeiro', 'eliminar', 'antes', 'a...
9206    ['porque', 'morning', 'call', 'aparecer', 'spo...
Name: texto_tratado, Length: 9207, dtype: object

In [38]:
# Supondo que seu DataFrame seja chamado de df e a coluna seja 'texto_tratado'
df['texto_tratado'] = df['texto_tratado'].str.replace("'", "")
df['texto_tratado']

0       [alvarez, marsal, estar, conosco, sportainmet,...
1       [btgpactual, with, makerepost, entender, impac...
2                                   [minuto, touro, ouro]
3                                                   [sim]
4           [querer, saber, banking, próprio, administro]
                              ...                        
9202                              [excelente, explicação]
9203                      [atendar, telefone, amor, deus]
9204    [saber, qual, grande, fiis, mercado, selecione...
9205    [erro, financeiro, eliminar, antes, ano, _, pa...
9206    [porque, morning, call, aparecer, spotify, atu...
Name: texto_tratado, Length: 9207, dtype: object

# 10. Word2Vec com CBOW

## Estruturação

In [39]:
df

,autor,sentimento,texto_tratado
0,winthegame_of,1,"[alvarez, marsal, estar, conosco, sportainmet,..."
1,marta_bego,1,"[btgpactual, with, makerepost, entender, impac..."
2,lmviapiana,2,"[minuto, touro, ouro]"
3,vanilson_dos,1,[sim]
4,ricktolledo,2,"[querer, saber, banking, próprio, administro]"
...,...,...,...
9202,perspectiveinvestimentos,2,"[excelente, explicação]"
9203,eduardocolares,2,"[atendar, telefone, amor, deus]"
9204,danielucm,2,"[saber, qual, grande, fiis, mercado, selecione..."
9205,amgcapitalinvest,1,"[erro, financeiro, eliminar, antes, ano, _, pa..."


In [40]:
cbow = '/content/drive/MyDrive/Módulo 6/Semana 5/cbow_s50/cbow_s50.txt'

In [41]:
model_cbow = KeyedVectors.load_word2vec_format(cbow)

## Teste isolado

In [42]:
# Testando o word2vec
wordvec_test = model_cbow['projeto']

wordvec_test

array([-0.074174, -0.152088,  0.086627, -0.224567,  0.362562,  0.130683,
       -0.089179, -0.086973,  0.309501,  0.004112, -0.308202,  0.351789,
       -0.477863,  0.050276,  0.213283,  0.159895, -0.285545, -0.08832 ,
       -0.015449,  0.014816, -0.613861,  0.502556,  0.021688,  0.369492,
        0.280691,  0.016868,  0.105584, -0.180754, -0.078456,  0.148032,
        0.36293 , -0.011634,  0.412191, -0.009049,  0.010404,  0.131242,
       -0.032483, -0.133067, -0.063802,  0.434015, -0.214768, -0.072132,
        0.045601, -0.368866,  0.502808,  0.048293, -0.254894,  0.142581,
       -0.075066,  0.015646], dtype=float32)

## Definição de função

In [43]:
def create_sentence_vector(model, df):
    sentence_table = []
    for sentence in df['texto_tratado']:
        word_vectors = [model[word] for word in sentence if word in model]
        if len(word_vectors) > 0:
            sentence_vector = sum(word_vectors) / len(word_vectors)
        else:
            sentence_vector = [None] * 100  # Cria uma lista de 100 elementos None
        sentence_table.append((sentence, *sentence_vector[:50]))  # Adiciona apenas os primeiros 50 elementos do vetor

    column_labels = ['Frase']
    for i in range(50):
        column_labels.append(f'Vetor{i+1}')
    df_vec = pd.DataFrame(sentence_table, columns=column_labels)

    df["sentimentoNumerico"] = df["sentimento"].replace({'NEGATIVE': -1, 'POSITIVE': 1, 'NEUTRAL': 0})

    # Definir o índice do DataFrame df_vec como o mesmo índice de df_processada['sentimentoNumerico']
    df_vec.set_index(df["sentimentoNumerico"].index, inplace=True)

    df_vec['sentimento'] = df["sentimentoNumerico"]
    df_vec = df_vec.dropna()

    return df_vec

## Teste de funções

In [44]:
df_vec = create_sentence_vector(model_cbow, df)
df_vec

,Frase,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,Vetor8,Vetor9,...,Vetor42,Vetor43,Vetor44,Vetor45,Vetor46,Vetor47,Vetor48,Vetor49,Vetor50,sentimento
0,"[alvarez, marsal, estar, conosco, sportainmet,...",0.213634,-0.129877,0.241601,-0.075002,-0.015629,0.206194,0.072658,0.055472,0.061554,...,0.024361,-0.111328,0.157674,0.094309,-0.047458,0.157365,-0.033920,0.022211,0.182153,1
1,"[btgpactual, with, makerepost, entender, impac...",0.222697,-0.124886,0.213157,-0.059091,-0.010530,0.201566,0.071898,0.033920,0.059524,...,0.008988,-0.079109,0.159296,0.085387,-0.008607,0.158519,-0.022680,0.031107,0.189521,1
2,"[minuto, touro, ouro]",0.265227,-0.068285,0.152235,-0.044329,-0.102729,0.141353,0.092800,0.113174,0.015783,...,0.078032,-0.202677,0.155750,0.062291,0.007038,0.134573,0.014635,0.034189,0.345674,2
3,[sim],0.166258,-0.029796,0.204045,-0.297490,0.046077,0.140763,0.035251,-0.174491,0.211817,...,0.065839,-0.092451,0.308218,-0.034692,-0.032851,-0.028724,-0.068701,0.011158,0.258413,1
4,"[querer, saber, banking, próprio, administro]",0.187512,-0.183612,0.300155,-0.052422,-0.034717,0.232278,0.058778,0.084289,0.088006,...,0.097538,-0.161461,0.196748,0.088577,-0.080884,0.167507,-0.049984,-0.000942,0.187811,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9202,"[excelente, explicação]",0.190917,-0.133475,0.241675,-0.053180,0.067256,0.201138,0.034109,-0.078718,-0.066131,...,-0.082151,0.016113,0.154861,0.068700,-0.004302,0.079717,-0.028388,-0.017448,0.188785,2
9203,"[atendar, telefone, amor, deus]",0.188641,-0.119377,0.199339,-0.105448,0.023176,0.178837,0.069476,-0.004494,0.034710,...,0.034035,-0.126673,0.165176,0.080313,-0.024160,0.118848,-0.003502,0.087053,0.215656,2
9204,"[saber, qual, grande, fiis, mercado, selecione...",0.215474,-0.137852,0.223206,-0.072183,-0.013213,0.205186,0.063497,0.039164,0.070273,...,0.034706,-0.097793,0.177275,0.090335,-0.047405,0.154374,-0.028906,0.023713,0.179591,2
9205,"[erro, financeiro, eliminar, antes, ano, _, pa...",0.219393,-0.129317,0.239226,-0.064735,-0.025696,0.224218,0.070732,0.042386,0.040706,...,0.025414,-0.108338,0.160880,0.092846,-0.032266,0.151619,-0.023750,0.028080,0.191956,1


In [45]:
#df_vec.to_csv('Word2Vec_Cbow_modelo_treinado',encoding='utf-8', index=False, header=True)

# 11. Naive Bayes + Word2Vec com CBOW

In [46]:
label = preprocessing.LabelEncoder()

In [47]:
label.fit(df_vec['sentimento'])
df_vec['sentimento'] = label.transform(df_vec['sentimento'])

In [48]:
df_vec = df_vec.dropna()
df_vec

,Frase,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,Vetor8,Vetor9,...,Vetor42,Vetor43,Vetor44,Vetor45,Vetor46,Vetor47,Vetor48,Vetor49,Vetor50,sentimento
0,"[alvarez, marsal, estar, conosco, sportainmet,...",0.213634,-0.129877,0.241601,-0.075002,-0.015629,0.206194,0.072658,0.055472,0.061554,...,0.024361,-0.111328,0.157674,0.094309,-0.047458,0.157365,-0.033920,0.022211,0.182153,1
1,"[btgpactual, with, makerepost, entender, impac...",0.222697,-0.124886,0.213157,-0.059091,-0.010530,0.201566,0.071898,0.033920,0.059524,...,0.008988,-0.079109,0.159296,0.085387,-0.008607,0.158519,-0.022680,0.031107,0.189521,1
2,"[minuto, touro, ouro]",0.265227,-0.068285,0.152235,-0.044329,-0.102729,0.141353,0.092800,0.113174,0.015783,...,0.078032,-0.202677,0.155750,0.062291,0.007038,0.134573,0.014635,0.034189,0.345674,2
3,[sim],0.166258,-0.029796,0.204045,-0.297490,0.046077,0.140763,0.035251,-0.174491,0.211817,...,0.065839,-0.092451,0.308218,-0.034692,-0.032851,-0.028724,-0.068701,0.011158,0.258413,1
4,"[querer, saber, banking, próprio, administro]",0.187512,-0.183612,0.300155,-0.052422,-0.034717,0.232278,0.058778,0.084289,0.088006,...,0.097538,-0.161461,0.196748,0.088577,-0.080884,0.167507,-0.049984,-0.000942,0.187811,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9202,"[excelente, explicação]",0.190917,-0.133475,0.241675,-0.053180,0.067256,0.201138,0.034109,-0.078718,-0.066131,...,-0.082151,0.016113,0.154861,0.068700,-0.004302,0.079717,-0.028388,-0.017448,0.188785,2
9203,"[atendar, telefone, amor, deus]",0.188641,-0.119377,0.199339,-0.105448,0.023176,0.178837,0.069476,-0.004494,0.034710,...,0.034035,-0.126673,0.165176,0.080313,-0.024160,0.118848,-0.003502,0.087053,0.215656,2
9204,"[saber, qual, grande, fiis, mercado, selecione...",0.215474,-0.137852,0.223206,-0.072183,-0.013213,0.205186,0.063497,0.039164,0.070273,...,0.034706,-0.097793,0.177275,0.090335,-0.047405,0.154374,-0.028906,0.023713,0.179591,2
9205,"[erro, financeiro, eliminar, antes, ano, _, pa...",0.219393,-0.129317,0.239226,-0.064735,-0.025696,0.224218,0.070732,0.042386,0.040706,...,0.025414,-0.108338,0.160880,0.092846,-0.032266,0.151619,-0.023750,0.028080,0.191956,1


## Separando Treino e Teste

In [49]:
target = df_vec['sentimento']

In [50]:
feature = df_vec.iloc[:,1:50]

In [51]:
feature

,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,Vetor8,Vetor9,Vetor10,...,Vetor40,Vetor41,Vetor42,Vetor43,Vetor44,Vetor45,Vetor46,Vetor47,Vetor48,Vetor49
0,0.213634,-0.129877,0.241601,-0.075002,-0.015629,0.206194,0.072658,0.055472,0.061554,0.170172,...,0.074627,-0.270438,0.024361,-0.111328,0.157674,0.094309,-0.047458,0.157365,-0.033920,0.022211
1,0.222697,-0.124886,0.213157,-0.059091,-0.010530,0.201566,0.071898,0.033920,0.059524,0.164536,...,0.086584,-0.301574,0.008988,-0.079109,0.159296,0.085387,-0.008607,0.158519,-0.022680,0.031107
2,0.265227,-0.068285,0.152235,-0.044329,-0.102729,0.141353,0.092800,0.113174,0.015783,0.202198,...,-0.008447,-0.193025,0.078032,-0.202677,0.155750,0.062291,0.007038,0.134573,0.014635,0.034189
3,0.166258,-0.029796,0.204045,-0.297490,0.046077,0.140763,0.035251,-0.174491,0.211817,0.288314,...,0.183434,-0.415105,0.065839,-0.092451,0.308218,-0.034692,-0.032851,-0.028724,-0.068701,0.011158
4,0.187512,-0.183612,0.300155,-0.052422,-0.034717,0.232278,0.058778,0.084289,0.088006,0.148424,...,0.046667,-0.247744,0.097538,-0.161461,0.196748,0.088577,-0.080884,0.167507,-0.049984,-0.000942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9202,0.190917,-0.133475,0.241675,-0.053180,0.067256,0.201138,0.034109,-0.078718,-0.066131,0.187608,...,0.014565,-0.321192,-0.082151,0.016113,0.154861,0.068700,-0.004302,0.079717,-0.028388,-0.017448
9203,0.188641,-0.119377,0.199339,-0.105448,0.023176,0.178837,0.069476,-0.004494,0.034710,0.150081,...,0.071114,-0.194663,0.034035,-0.126673,0.165176,0.080313,-0.024160,0.118848,-0.003502,0.087053
9204,0.215474,-0.137852,0.223206,-0.072183,-0.013213,0.205186,0.063497,0.039164,0.070273,0.172185,...,0.081652,-0.300155,0.034706,-0.097793,0.177275,0.090335,-0.047405,0.154374,-0.028906,0.023713
9205,0.219393,-0.129317,0.239226,-0.064735,-0.025696,0.224218,0.070732,0.042386,0.040706,0.154811,...,0.063415,-0.258450,0.025414,-0.108338,0.160880,0.092846,-0.032266,0.151619,-0.023750,0.028080


In [52]:
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.2, random_state=42)

## Avaliação do modelo 

In [53]:
clf = GaussianNB()

clf = clf.fit(X_train,y_train.values.ravel())

Y_pred = clf.predict(X_test)

print(classification_report(y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.29      0.74      0.42       386
           1       0.74      0.46      0.56       844
           2       0.34      0.20      0.25       612

    accuracy                           0.43      1842
   macro avg       0.46      0.46      0.41      1842
weighted avg       0.51      0.43      0.43      1842



In [54]:
acc_score = accuracy_score(y_test, Y_pred)
format_output = "{:.2%}".format(acc_score)
print("Precisão final de :",format_output) 

Precisão final de : 42.94%


# 12. Word2Vec com embedding layer

## Definição de função

In [55]:
from gensim.models import Word2Vec

# Função que treina o modelo Word2Vec no corpus do dataframe
def train_word2vec(df, column_name):
    # Obtém as frases tokenizadas
    sentences = df[column_name].tolist()
    
    # Treina o modelo Word2Vec
    model = Word2Vec(sentences, min_count=1)
    
    return model

In [56]:
# Função que define os vetores para cada palavra do vocabulario
def get_word_vectors(model, sentence):
    vectors = []
    for word in sentence:
        if word in model.wv:
            vectors.append(model.wv[word]) # Append na lista de vetores
    if vectors:
        return np.sum(vectors, axis=0)/len(sentence) # Soma dos vetores para cada frase
    else:
        return np.zeros(model.vector_size)

# Criação do dataframe de vetores para cada frase
def create_word2vec_dataframe(df, column_name, model):
    sentences = df[column_name].tolist()
    vectors = [get_word_vectors(model, sentence) for sentence in sentences] # Itera para cada frase um vetor
    # Criação do dataframe
    df_vectors = pd.DataFrame(vectors, columns=[f"Vetor{i}" for i in range(model.vector_size)])
    df_word2vec = pd.concat([df, df_vectors], axis=1)
    return df_word2vec

## Teste de funções

In [57]:
model = train_word2vec(df, 'texto_tratado')

In [58]:
df_word2vec = create_word2vec_dataframe(df,'texto_tratado', model)
df_word2vec

,autor,sentimento,texto_tratado,sentimentoNumerico,Vetor0,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,...,Vetor90,Vetor91,Vetor92,Vetor93,Vetor94,Vetor95,Vetor96,Vetor97,Vetor98,Vetor99
0,winthegame_of,1,"[alvarez, marsal, estar, conosco, sportainmet,...",1,0.158088,0.081801,-0.312730,-0.085220,0.155300,0.543125,...,-0.114124,0.120844,0.067580,0.401354,-0.254166,-0.030406,-0.296194,0.109436,0.088832,0.020928
1,marta_bego,1,"[btgpactual, with, makerepost, entender, impac...",1,0.158730,0.082317,-0.308764,-0.072660,0.153781,0.547262,...,-0.111215,0.123652,0.059257,0.368752,-0.267438,-0.011545,-0.306010,0.123276,0.078062,0.035625
2,lmviapiana,2,"[minuto, touro, ouro]",2,0.140351,0.081411,-0.393997,-0.056730,0.202745,0.570919,...,-0.126200,0.256177,0.051310,0.434570,-0.190311,0.038305,-0.417735,0.110342,0.141120,0.026097
3,vanilson_dos,1,[sim],1,0.260669,0.153282,-0.226586,0.344876,0.211045,0.475191,...,-0.137162,-0.107625,0.023879,0.353527,-0.210970,0.019192,-0.336056,0.178022,0.121753,-0.149758
4,ricktolledo,2,"[querer, saber, banking, próprio, administro]",2,0.110473,0.088539,-0.319897,-0.013789,0.166744,0.486413,...,-0.126261,0.085323,0.070803,0.343654,-0.236273,-0.014238,-0.261656,0.118241,0.105460,0.109313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9202,perspectiveinvestimentos,2,"[excelente, explicação]",2,0.233469,0.047241,-0.357967,-0.142721,0.136204,0.589766,...,-0.071484,0.028768,0.031334,0.435927,-0.332207,0.035537,-0.285489,-0.045111,0.149235,0.009134
9203,eduardocolares,2,"[atendar, telefone, amor, deus]",2,0.126397,0.123738,-0.297996,-0.157727,0.173517,0.595546,...,-0.134568,0.122979,0.080740,0.366018,-0.301596,-0.011802,-0.297539,0.170522,0.084630,0.083352
9204,danielucm,2,"[saber, qual, grande, fiis, mercado, selecione...",2,0.146785,0.057451,-0.312274,-0.135915,0.168704,0.559119,...,-0.076423,0.116622,0.049559,0.435275,-0.285202,-0.010786,-0.302765,0.065949,0.117888,0.060783
9205,amgcapitalinvest,1,"[erro, financeiro, eliminar, antes, ano, _, pa...",1,0.164539,0.072891,-0.339579,-0.127373,0.165823,0.575756,...,-0.105292,0.142347,0.054528,0.421809,-0.286428,-0.037198,-0.348306,0.102846,0.095914,0.053667


In [59]:
df_word2vec = df_word2vec.drop(columns=['autor', 'sentimento'])
df_word2vec

,texto_tratado,sentimentoNumerico,Vetor0,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,...,Vetor90,Vetor91,Vetor92,Vetor93,Vetor94,Vetor95,Vetor96,Vetor97,Vetor98,Vetor99
0,"[alvarez, marsal, estar, conosco, sportainmet,...",1,0.158088,0.081801,-0.312730,-0.085220,0.155300,0.543125,-0.118902,-0.302126,...,-0.114124,0.120844,0.067580,0.401354,-0.254166,-0.030406,-0.296194,0.109436,0.088832,0.020928
1,"[btgpactual, with, makerepost, entender, impac...",1,0.158730,0.082317,-0.308764,-0.072660,0.153781,0.547262,-0.142779,-0.308808,...,-0.111215,0.123652,0.059257,0.368752,-0.267438,-0.011545,-0.306010,0.123276,0.078062,0.035625
2,"[minuto, touro, ouro]",2,0.140351,0.081411,-0.393997,-0.056730,0.202745,0.570919,-0.202274,-0.339550,...,-0.126200,0.256177,0.051310,0.434570,-0.190311,0.038305,-0.417735,0.110342,0.141120,0.026097
3,[sim],1,0.260669,0.153282,-0.226586,0.344876,0.211045,0.475191,-0.083177,-0.243225,...,-0.137162,-0.107625,0.023879,0.353527,-0.210970,0.019192,-0.336056,0.178022,0.121753,-0.149758
4,"[querer, saber, banking, próprio, administro]",2,0.110473,0.088539,-0.319897,-0.013789,0.166744,0.486413,-0.127889,-0.267826,...,-0.126261,0.085323,0.070803,0.343654,-0.236273,-0.014238,-0.261656,0.118241,0.105460,0.109313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9202,"[excelente, explicação]",2,0.233469,0.047241,-0.357967,-0.142721,0.136204,0.589766,-0.086757,-0.312808,...,-0.071484,0.028768,0.031334,0.435927,-0.332207,0.035537,-0.285489,-0.045111,0.149235,0.009134
9203,"[atendar, telefone, amor, deus]",2,0.126397,0.123738,-0.297996,-0.157727,0.173517,0.595546,-0.194869,-0.324687,...,-0.134568,0.122979,0.080740,0.366018,-0.301596,-0.011802,-0.297539,0.170522,0.084630,0.083352
9204,"[saber, qual, grande, fiis, mercado, selecione...",2,0.146785,0.057451,-0.312274,-0.135915,0.168704,0.559119,-0.114752,-0.289645,...,-0.076423,0.116622,0.049559,0.435275,-0.285202,-0.010786,-0.302765,0.065949,0.117888,0.060783
9205,"[erro, financeiro, eliminar, antes, ano, _, pa...",1,0.164539,0.072891,-0.339579,-0.127373,0.165823,0.575756,-0.127015,-0.334934,...,-0.105292,0.142347,0.054528,0.421809,-0.286428,-0.037198,-0.348306,0.102846,0.095914,0.053667


# 13. Naive Bayes + Word2Vec com embedding layer

In [60]:
df_word2vec = df_word2vec.dropna()
df_word2vec

,texto_tratado,sentimentoNumerico,Vetor0,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,...,Vetor90,Vetor91,Vetor92,Vetor93,Vetor94,Vetor95,Vetor96,Vetor97,Vetor98,Vetor99
0,"[alvarez, marsal, estar, conosco, sportainmet,...",1,0.158088,0.081801,-0.312730,-0.085220,0.155300,0.543125,-0.118902,-0.302126,...,-0.114124,0.120844,0.067580,0.401354,-0.254166,-0.030406,-0.296194,0.109436,0.088832,0.020928
1,"[btgpactual, with, makerepost, entender, impac...",1,0.158730,0.082317,-0.308764,-0.072660,0.153781,0.547262,-0.142779,-0.308808,...,-0.111215,0.123652,0.059257,0.368752,-0.267438,-0.011545,-0.306010,0.123276,0.078062,0.035625
2,"[minuto, touro, ouro]",2,0.140351,0.081411,-0.393997,-0.056730,0.202745,0.570919,-0.202274,-0.339550,...,-0.126200,0.256177,0.051310,0.434570,-0.190311,0.038305,-0.417735,0.110342,0.141120,0.026097
3,[sim],1,0.260669,0.153282,-0.226586,0.344876,0.211045,0.475191,-0.083177,-0.243225,...,-0.137162,-0.107625,0.023879,0.353527,-0.210970,0.019192,-0.336056,0.178022,0.121753,-0.149758
4,"[querer, saber, banking, próprio, administro]",2,0.110473,0.088539,-0.319897,-0.013789,0.166744,0.486413,-0.127889,-0.267826,...,-0.126261,0.085323,0.070803,0.343654,-0.236273,-0.014238,-0.261656,0.118241,0.105460,0.109313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9202,"[excelente, explicação]",2,0.233469,0.047241,-0.357967,-0.142721,0.136204,0.589766,-0.086757,-0.312808,...,-0.071484,0.028768,0.031334,0.435927,-0.332207,0.035537,-0.285489,-0.045111,0.149235,0.009134
9203,"[atendar, telefone, amor, deus]",2,0.126397,0.123738,-0.297996,-0.157727,0.173517,0.595546,-0.194869,-0.324687,...,-0.134568,0.122979,0.080740,0.366018,-0.301596,-0.011802,-0.297539,0.170522,0.084630,0.083352
9204,"[saber, qual, grande, fiis, mercado, selecione...",2,0.146785,0.057451,-0.312274,-0.135915,0.168704,0.559119,-0.114752,-0.289645,...,-0.076423,0.116622,0.049559,0.435275,-0.285202,-0.010786,-0.302765,0.065949,0.117888,0.060783
9205,"[erro, financeiro, eliminar, antes, ano, _, pa...",1,0.164539,0.072891,-0.339579,-0.127373,0.165823,0.575756,-0.127015,-0.334934,...,-0.105292,0.142347,0.054528,0.421809,-0.286428,-0.037198,-0.348306,0.102846,0.095914,0.053667


## Separando Treino e Teste

In [61]:
target = df_word2vec['sentimentoNumerico']

In [62]:
feature = df_word2vec.iloc[:,2:102]

In [63]:
feature

,Vetor0,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,Vetor8,Vetor9,...,Vetor90,Vetor91,Vetor92,Vetor93,Vetor94,Vetor95,Vetor96,Vetor97,Vetor98,Vetor99
0,0.158088,0.081801,-0.312730,-0.085220,0.155300,0.543125,-0.118902,-0.302126,-0.130301,0.555786,...,-0.114124,0.120844,0.067580,0.401354,-0.254166,-0.030406,-0.296194,0.109436,0.088832,0.020928
1,0.158730,0.082317,-0.308764,-0.072660,0.153781,0.547262,-0.142779,-0.308808,-0.114442,0.558739,...,-0.111215,0.123652,0.059257,0.368752,-0.267438,-0.011545,-0.306010,0.123276,0.078062,0.035625
2,0.140351,0.081411,-0.393997,-0.056730,0.202745,0.570919,-0.202274,-0.339550,-0.203485,0.718141,...,-0.126200,0.256177,0.051310,0.434570,-0.190311,0.038305,-0.417735,0.110342,0.141120,0.026097
3,0.260669,0.153282,-0.226586,0.344876,0.211045,0.475191,-0.083177,-0.243225,-0.219113,0.627887,...,-0.137162,-0.107625,0.023879,0.353527,-0.210970,0.019192,-0.336056,0.178022,0.121753,-0.149758
4,0.110473,0.088539,-0.319897,-0.013789,0.166744,0.486413,-0.127889,-0.267826,-0.172174,0.542878,...,-0.126261,0.085323,0.070803,0.343654,-0.236273,-0.014238,-0.261656,0.118241,0.105460,0.109313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9202,0.233469,0.047241,-0.357967,-0.142721,0.136204,0.589766,-0.086757,-0.312808,-0.238086,0.539663,...,-0.071484,0.028768,0.031334,0.435927,-0.332207,0.035537,-0.285489,-0.045111,0.149235,0.009134
9203,0.126397,0.123738,-0.297996,-0.157727,0.173517,0.595546,-0.194869,-0.324687,-0.203900,0.632138,...,-0.134568,0.122979,0.080740,0.366018,-0.301596,-0.011802,-0.297539,0.170522,0.084630,0.083352
9204,0.146785,0.057451,-0.312274,-0.135915,0.168704,0.559119,-0.114752,-0.289645,-0.157744,0.541534,...,-0.076423,0.116622,0.049559,0.435275,-0.285202,-0.010786,-0.302765,0.065949,0.117888,0.060783
9205,0.164539,0.072891,-0.339579,-0.127373,0.165823,0.575756,-0.127015,-0.334934,-0.135407,0.574449,...,-0.105292,0.142347,0.054528,0.421809,-0.286428,-0.037198,-0.348306,0.102846,0.095914,0.053667


In [64]:
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.2, random_state=42)

## Avaliação do modelo 

In [65]:
clf = GaussianNB()

clf = clf.fit(X_train,y_train.values.ravel())

Y_pred = clf.predict(X_test)

print(classification_report(y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.29      0.83      0.43       386
           1       0.78      0.46      0.58       844
           2       0.39      0.13      0.20       612

    accuracy                           0.43      1842
   macro avg       0.48      0.48      0.40      1842
weighted avg       0.54      0.43      0.42      1842



In [66]:
acc_score = accuracy_score(y_test, Y_pred)
format_output = "{:.2%}".format(acc_score)
print("Precisão final de :",format_output) 

Precisão final de : 43.05%


# Rede Neural - Sequência de palavras

## Teste isolado

In [67]:
! pip install pad_sequences

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pad_sequences: filename=pad_sequences-0.6.1-py3-none-any.whl size=10199 sha256=fbb957f061a1a3dd14367f406d984eb166144fa54d73cc75032061ef32304479
  Stored in directory: /root/.cache/pip/wheels/48/9d/22/0a6305b87a9cc46ccc032060a041c3b59f39ac462f7358997e
Successfully built pad_sequences


In [68]:
from keras.preprocessing.text import Tokenizer
#from keras_preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences

citacao = [
    "Estudo no Inteli e escolhi o curso de Sistemas de Informação",
    "Estamos fazendo um projeto para o BTG",
    "Estamos no segundo ano de graduação",
    "Somos do grupo BTG3"
]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(citacao)
citacao_token = tokenizer.texts_to_sequences(citacao)

max_length = max([len(z) for z in citacao_token]) + 1
citacao_pad = pad_sequences(citacao_token, maxlen=max_length, padding="post")

## Construção da rede neural + resultados - base tratada

A rede neural abaixo foi desenvolvida com o tutorial mencionado no ínicio do notebook, que, nesse caso o dataframe utilizado é um arquivo com a base que já passou pelo pré processamento. 

### Dataset já treinado

In [69]:
teste_rede_neural = pd.read_csv("/content/drive/MyDrive/Módulo 6/Sprint 3 - Projeto/nova_base_tratada (1)")

In [70]:
teste_rede_neural

,Unnamed: 0,autor,sentimento,texto_tratado
0,0,winthegame_of,1,"['alvarez', 'marsal', 'estar', 'conosco', 'spo..."
1,1,marta_bego,1,"['btgpactual', 'with', 'makerepost', 'entender..."
2,2,lmviapiana,2,"['minuto', 'touro', 'ouro']"
3,3,vanilson_dos,1,['sim']
4,4,ricktolledo,2,"['querer', 'saber', 'banking', 'próprio', 'adm..."
...,...,...,...,...
9202,9472,perspectiveinvestimentos,2,"['excelente', 'explicação']"
9203,9473,eduardocolares,2,"['atendar', 'telefone', 'amor', 'deus']"
9204,9474,danielucm,2,"['saber', 'qual', 'grande', 'fiis', 'mercado',..."
9205,9475,amgcapitalinvest,1,"['erro', 'financeiro', 'eliminar', 'antes', 'a..."


### Separação de treino e teste

In [71]:
x, y = teste_rede_neural["texto_tratado"], teste_rede_neural["sentimento"]

labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

words = ["o", "ao", 'aos', 'os', 'a', 'as', 'e', 'um', 'uma', 
        'ele', 'ela', 'eles', 'elas', 'do', 'da', 'dos', 'das', 
        'de', 'no', 'na', 'nos', 'nas', 'pelo', 'pela', 'pelos', 
        'pelas', 'num', 'numa', 'nuns', 'numas', 'dum', 'duma', 
        'duns', 'dumas']

x_filter = []

for title in x:
  for word in words:
    title = title.replace(word, '')
  x_filter.append(title)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_filter)

vocab = len(tokenizer.word_docs) + 1

x_filter = tokenizer.texts_to_sequences(x_filter)

max_length = max([len(z) for z in x_filter])
x_filter = pad_sequences(x_filter, maxlen=max_length, padding='post')

x_train, x_test, y_train, y_test = train_test_split(x_filter, y, test_size=0.33)

print("Tamanho de x:", len(x_filter))
print("Tamanho de y:", len(y))


Tamanho de x: 9207
Tamanho de y: 9207


### Criação do modelo

In [72]:
model = Sequential()
model.add(Embedding(input_dim=vocab, output_dim=80, input_length=max_length, trainable = True))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.3))
model.add(Dense(units = 3, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

mc = ModelCheckpoint('weight.best.hdf5', monitor='val_acc', save_best_only=True, mode='max')

model.fit(x_train, y_train, validation_data = (x_test, y_test), batch_size = 32, epochs = 5, callbacks = [mc])

print(model.evaluate(x_test, y_test))

Epoch 1/5
193/193 [==============================] - ETA: 0s - loss: 0.9919 - accuracy: 0.5269

193/193 [==============================] - 7s 31ms/step - loss: 0.9919 - accuracy: 0.5269 - val_loss: 0.9114 - val_accuracy: 0.5518
Epoch 2/5
193/193 [==============================] - ETA: 0s - loss: 0.8214 - accuracy: 0.6247

193/193 [==============================] - 9s 46ms/step - loss: 0.8214 - accuracy: 0.6247 - val_loss: 0.7944 - val_accuracy: 0.6466
Epoch 3/5
193/193 [==============================] - ETA: 0s - loss: 0.6955 - accuracy: 0.7280

193/193 [==============================] - 7s 38ms/step - loss: 0.6955 - accuracy: 0.7280 - val_loss: 0.7191 - val_accuracy: 0.7022
Epoch 4/5
193/193 [==============================] - ETA: 0s - loss: 0.5963 - accuracy: 0.7887

193/193 [==============================] - 9s 46ms/step - loss: 0.5963 - accuracy: 0.7887 - val_loss: 0.6739 - val_accuracy: 0.7183
Epoch 5/5
193/193 [==============================] - ETA: 0s - loss: 0.5136 - accuracy: 0.8281

95/95 [==============================] - 0s 2ms/step - loss: 0.6445 - accuracy: 0.7279
[0.644501805305481, 0.7278710007667542]


In [73]:
# Avaliação do modelo
loss, accuracy = model.evaluate(x_test, y_test)
print('Loss:', loss)
print('Accuracy:', accuracy)

95/95 [==============================] - 0s 2ms/step - loss: 0.6445 - accuracy: 0.7279
Loss: 0.644501805305481
Accuracy: 0.7278710007667542


### Exportação com a biblioteca pickle

In [74]:
with open('redeNeural_base.pkl', 'wb') as arquivo:
    pickle.dump(model, arquivo)
with open('redeNeural_base.pkl', 'rb') as arquivo:
    modelo_redeNeural_base = pickle.load(arquivo)

## Construção da rede neural + resultados - word2vec com cbow

A rede neural abaixo foi desenvolvida com o tutorial mencionado no ínicio do notebook, que, nesse caso o dataframe utilizado é a variável do Word2Vec com o CBoW. 

### Dataframe do word2vec com o cbow


In [75]:
df_vec

,Frase,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,Vetor8,Vetor9,...,Vetor42,Vetor43,Vetor44,Vetor45,Vetor46,Vetor47,Vetor48,Vetor49,Vetor50,sentimento
0,"[alvarez, marsal, estar, conosco, sportainmet,...",0.213634,-0.129877,0.241601,-0.075002,-0.015629,0.206194,0.072658,0.055472,0.061554,...,0.024361,-0.111328,0.157674,0.094309,-0.047458,0.157365,-0.033920,0.022211,0.182153,1
1,"[btgpactual, with, makerepost, entender, impac...",0.222697,-0.124886,0.213157,-0.059091,-0.010530,0.201566,0.071898,0.033920,0.059524,...,0.008988,-0.079109,0.159296,0.085387,-0.008607,0.158519,-0.022680,0.031107,0.189521,1
2,"[minuto, touro, ouro]",0.265227,-0.068285,0.152235,-0.044329,-0.102729,0.141353,0.092800,0.113174,0.015783,...,0.078032,-0.202677,0.155750,0.062291,0.007038,0.134573,0.014635,0.034189,0.345674,2
3,[sim],0.166258,-0.029796,0.204045,-0.297490,0.046077,0.140763,0.035251,-0.174491,0.211817,...,0.065839,-0.092451,0.308218,-0.034692,-0.032851,-0.028724,-0.068701,0.011158,0.258413,1
4,"[querer, saber, banking, próprio, administro]",0.187512,-0.183612,0.300155,-0.052422,-0.034717,0.232278,0.058778,0.084289,0.088006,...,0.097538,-0.161461,0.196748,0.088577,-0.080884,0.167507,-0.049984,-0.000942,0.187811,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9202,"[excelente, explicação]",0.190917,-0.133475,0.241675,-0.053180,0.067256,0.201138,0.034109,-0.078718,-0.066131,...,-0.082151,0.016113,0.154861,0.068700,-0.004302,0.079717,-0.028388,-0.017448,0.188785,2
9203,"[atendar, telefone, amor, deus]",0.188641,-0.119377,0.199339,-0.105448,0.023176,0.178837,0.069476,-0.004494,0.034710,...,0.034035,-0.126673,0.165176,0.080313,-0.024160,0.118848,-0.003502,0.087053,0.215656,2
9204,"[saber, qual, grande, fiis, mercado, selecione...",0.215474,-0.137852,0.223206,-0.072183,-0.013213,0.205186,0.063497,0.039164,0.070273,...,0.034706,-0.097793,0.177275,0.090335,-0.047405,0.154374,-0.028906,0.023713,0.179591,2
9205,"[erro, financeiro, eliminar, antes, ano, _, pa...",0.219393,-0.129317,0.239226,-0.064735,-0.025696,0.224218,0.070732,0.042386,0.040706,...,0.025414,-0.108338,0.160880,0.092846,-0.032266,0.151619,-0.023750,0.028080,0.191956,1


### Separação treino e teste

In [76]:
x, y = df_vec["Frase"], df_vec["sentimento"]

labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

words = ["o", "ao", 'aos', 'os', 'a', 'as', 'e', 'um', 'uma', 
        'ele', 'ela', 'eles', 'elas', 'do', 'da', 'dos', 'das', 
        'de', 'no', 'na', 'nos', 'nas', 'pelo', 'pela', 'pelos', 
        'pelas', 'num', 'numa', 'nuns', 'numas', 'dum', 'duma', 
        'duns', 'dumas']

x_filter = []

for title in x:
  for word in words:
    title = title.replace(word, '')
  x_filter.append(title)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_filter)

vocab = len(tokenizer.word_docs) + 1

x_filter = tokenizer.texts_to_sequences(x_filter)

max_length = max([len(z) for z in x_filter])
x_filter = pad_sequences(x_filter, maxlen=max_length, padding='post')

x_train, x_test, y_train, y_test = train_test_split(x_filter, y, test_size=0.33)

print("Tamanho de x:", len(x_filter))
print("Tamanho de y:", len(y))


Tamanho de x: 9207
Tamanho de y: 9207


### Criação do modelo

In [77]:
model = Sequential()
model.add(Embedding(input_dim=vocab, output_dim=80, input_length=max_length, trainable = True))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.3))
model.add(Dense(units = 3, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

mc = ModelCheckpoint('weight.best.hdf5', monitor='val_acc', save_best_only=True, mode='max')

model.fit(x_train, y_train, validation_data = (x_test, y_test), batch_size = 32, epochs = 5, callbacks = [mc])

print(model.evaluate(x_test, y_test))

Epoch 1/5
192/193 [============================>.] - ETA: 0s - loss: 1.0055 - accuracy: 0.5052

193/193 [==============================] - 7s 34ms/step - loss: 1.0050 - accuracy: 0.5057 - val_loss: 0.9044 - val_accuracy: 0.5758
Epoch 2/5
193/193 [==============================] - ETA: 0s - loss: 0.8279 - accuracy: 0.6433

193/193 [==============================] - 7s 34ms/step - loss: 0.8279 - accuracy: 0.6433 - val_loss: 0.7826 - val_accuracy: 0.6815
Epoch 3/5
192/193 [============================>.] - ETA: 0s - loss: 0.6973 - accuracy: 0.7383

193/193 [==============================] - 8s 41ms/step - loss: 0.6973 - accuracy: 0.7385 - val_loss: 0.7158 - val_accuracy: 0.6979
Epoch 4/5
192/193 [============================>.] - ETA: 0s - loss: 0.5986 - accuracy: 0.7884

193/193 [==============================] - 5s 28ms/step - loss: 0.5988 - accuracy: 0.7884 - val_loss: 0.6755 - val_accuracy: 0.7170
Epoch 5/5
191/193 [============================>.] - ETA: 0s - loss: 0.5183 - accuracy: 0.8159

95/95 [==============================] - 0s 3ms/step - loss: 0.6483 - accuracy: 0.7289
[0.6483069658279419, 0.7288581728935242]


In [78]:
# Avaliação do modelo
loss, accuracy = model.evaluate(x_test, y_test)
print('Loss:', loss)
print('Accuracy:', accuracy)

95/95 [==============================] - 0s 2ms/step - loss: 0.6483 - accuracy: 0.7289
Loss: 0.6483069658279419
Accuracy: 0.7288581728935242


### Exportação com a biblioteca pickle

In [79]:
with open('redeNeural_word2vec_cbow.pkl', 'wb') as arquivo:
    pickle.dump(model, arquivo)
with open('redeNeural_word2vec_cbow.pkl', 'rb') as arquivo:
    modelo_redeNeural_word2vec_cbow = pickle.load(arquivo)

## Construção da rede neural + resultados - word2vec com embedding layer

A rede neural abaixo foi desenvolvida com o tutorial mencionado no ínicio do notebook, que, nesse caso o dataframe utilizado é a variável do Word2Vec com Embedding Layer.

### Dataframe do word2vec com embedding layer

In [80]:
df_word2vec

,texto_tratado,sentimentoNumerico,Vetor0,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,...,Vetor90,Vetor91,Vetor92,Vetor93,Vetor94,Vetor95,Vetor96,Vetor97,Vetor98,Vetor99
0,"[alvarez, marsal, estar, conosco, sportainmet,...",1,0.158088,0.081801,-0.312730,-0.085220,0.155300,0.543125,-0.118902,-0.302126,...,-0.114124,0.120844,0.067580,0.401354,-0.254166,-0.030406,-0.296194,0.109436,0.088832,0.020928
1,"[btgpactual, with, makerepost, entender, impac...",1,0.158730,0.082317,-0.308764,-0.072660,0.153781,0.547262,-0.142779,-0.308808,...,-0.111215,0.123652,0.059257,0.368752,-0.267438,-0.011545,-0.306010,0.123276,0.078062,0.035625
2,"[minuto, touro, ouro]",2,0.140351,0.081411,-0.393997,-0.056730,0.202745,0.570919,-0.202274,-0.339550,...,-0.126200,0.256177,0.051310,0.434570,-0.190311,0.038305,-0.417735,0.110342,0.141120,0.026097
3,[sim],1,0.260669,0.153282,-0.226586,0.344876,0.211045,0.475191,-0.083177,-0.243225,...,-0.137162,-0.107625,0.023879,0.353527,-0.210970,0.019192,-0.336056,0.178022,0.121753,-0.149758
4,"[querer, saber, banking, próprio, administro]",2,0.110473,0.088539,-0.319897,-0.013789,0.166744,0.486413,-0.127889,-0.267826,...,-0.126261,0.085323,0.070803,0.343654,-0.236273,-0.014238,-0.261656,0.118241,0.105460,0.109313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9202,"[excelente, explicação]",2,0.233469,0.047241,-0.357967,-0.142721,0.136204,0.589766,-0.086757,-0.312808,...,-0.071484,0.028768,0.031334,0.435927,-0.332207,0.035537,-0.285489,-0.045111,0.149235,0.009134
9203,"[atendar, telefone, amor, deus]",2,0.126397,0.123738,-0.297996,-0.157727,0.173517,0.595546,-0.194869,-0.324687,...,-0.134568,0.122979,0.080740,0.366018,-0.301596,-0.011802,-0.297539,0.170522,0.084630,0.083352
9204,"[saber, qual, grande, fiis, mercado, selecione...",2,0.146785,0.057451,-0.312274,-0.135915,0.168704,0.559119,-0.114752,-0.289645,...,-0.076423,0.116622,0.049559,0.435275,-0.285202,-0.010786,-0.302765,0.065949,0.117888,0.060783
9205,"[erro, financeiro, eliminar, antes, ano, _, pa...",1,0.164539,0.072891,-0.339579,-0.127373,0.165823,0.575756,-0.127015,-0.334934,...,-0.105292,0.142347,0.054528,0.421809,-0.286428,-0.037198,-0.348306,0.102846,0.095914,0.053667


### Separação treino e teste


In [81]:
x, y = df_word2vec["texto_tratado"], df_word2vec["sentimentoNumerico"]

labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

words = ["o", "ao", 'aos', 'os', 'a', 'as', 'e', 'um', 'uma', 
        'ele', 'ela', 'eles', 'elas', 'do', 'da', 'dos', 'das', 
        'de', 'no', 'na', 'nos', 'nas', 'pelo', 'pela', 'pelos', 
        'pelas', 'num', 'numa', 'nuns', 'numas', 'dum', 'duma', 
        'duns', 'dumas']

x_filter = []

for title in x:
  for word in words:
    title = title.replace(word, '')
  x_filter.append(title)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_filter)

vocab = len(tokenizer.word_docs) + 1

x_filter = tokenizer.texts_to_sequences(x_filter)

max_length = max([len(z) for z in x_filter])
x_filter = pad_sequences(x_filter, maxlen=max_length, padding='post')

x_train, x_test, y_train, y_test = train_test_split(x_filter, y, test_size=0.33)

print("Tamanho de x:", len(x_filter))
print("Tamanho de y:", len(y))


Tamanho de x: 9207
Tamanho de y: 9207


### Criação do modelo

In [82]:
model = Sequential()
model.add(Embedding(input_dim=vocab, output_dim=80, input_length=max_length, trainable = True))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.3))
model.add(Dense(units = 3, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

mc = ModelCheckpoint('weight.best.hdf5', monitor='val_acc', save_best_only=True, mode='max')

model.fit(x_train, y_train, validation_data = (x_test, y_test), batch_size = 32, epochs = 5, callbacks = [mc])

print(model.evaluate(x_test, y_test))

Epoch 1/5
191/193 [============================>.] - ETA: 0s - loss: 1.0029 - accuracy: 0.5064

193/193 [==============================] - 6s 26ms/step - loss: 1.0024 - accuracy: 0.5063 - val_loss: 0.9181 - val_accuracy: 0.5587
Epoch 2/5
192/193 [============================>.] - ETA: 0s - loss: 0.8338 - accuracy: 0.6182

193/193 [==============================] - 4s 20ms/step - loss: 0.8338 - accuracy: 0.6182 - val_loss: 0.7980 - val_accuracy: 0.6456
Epoch 3/5
191/193 [============================>.] - ETA: 0s - loss: 0.7143 - accuracy: 0.7305

193/193 [==============================] - 4s 20ms/step - loss: 0.7144 - accuracy: 0.7301 - val_loss: 0.7292 - val_accuracy: 0.6963
Epoch 4/5
191/193 [============================>.] - ETA: 0s - loss: 0.6133 - accuracy: 0.7788

193/193 [==============================] - 5s 24ms/step - loss: 0.6129 - accuracy: 0.7785 - val_loss: 0.6840 - val_accuracy: 0.7101
Epoch 5/5
191/193 [============================>.] - ETA: 0s - loss: 0.5323 - accuracy: 0.8166

95/95 [==============================] - 0s 2ms/step - loss: 0.6551 - accuracy: 0.7213
[0.6551482081413269, 0.721289873123169]


In [83]:
# Avaliação do modelo
loss, accuracy = model.evaluate(x_test, y_test)
print('Loss:', loss)
print('Accuracy:', accuracy)

95/95 [==============================] - 0s 2ms/step - loss: 0.6551 - accuracy: 0.7213
Loss: 0.6551482081413269
Accuracy: 0.721289873123169


### Exportação com a biblioteca pickle 

In [84]:
with open('redeNeural_word2vec_embedding.pkl', 'wb') as arquivo:
    pickle.dump(model, arquivo)
with open('redeNeural_word2vec_embedding.pkl', 'rb') as arquivo:
    modelo_redeNeural_word2vec_embedding = pickle.load(arquivo)